UFC DATA SCAPER

Auther: Phillip Simmonds

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests as req
import datetime
import uuid

FIRST: COLLECT THE EVENTS DATA

In [ ]:
events_list_url = "http://ufcstats.com/statistics/events/completed?page=all"

In [ ]:
events_dictionary = {
    "eventurl" : [],
    "eventurlid" : [],
    "datestring" : [],
    "fighttitlefullname" : [],
    "fighttitlecategory" : [],
    "location" : []
    }

In [ ]:
def create_dict_for_events(fight_event):
    event_url = str(fight_event.find("a")).split('"')[3]
    event_url_id = event_url.split("/")[-1]
    date_string = fight_event.find("span").text.replace("\n","").strip()
    fight_title_full_name = fight_event.find("a").text.replace("\n","").strip()
    fight_title_category = fight_title_full_name.split(":")[0]
    location = fight_event.find_all("td")[-1].text.replace("\n","").strip()
    events_dictionary["eventurl"].append(event_url)
    events_dictionary["eventurlid"].append(event_url_id)
    events_dictionary["datestring"].append(date_string)
    events_dictionary["fighttitlefullname"].append(fight_title_full_name)
    events_dictionary["fighttitlecategory"].append(fight_title_category)
    events_dictionary["location"].append(location)


In [ ]:
events = BeautifulSoup(req.get(events_list_url).content, 'html.parser').find("table","b-statistics__table-events")
for fight_event in events.find_all("tr","b-statistics__table-row")[2:]:
    create_dict_for_events(fight_event)


In [ ]:
event_df = pd.DataFrame.from_dict(events_dictionary)
event_df.to_csv("extraction/events.csv")

SECOND: COLLECT THE FIGHT NIGHT DATA

In [ ]:
def sp(txt):
    return txt.text.strip()


In [ ]:
fighter_items = []

In [ ]:

for i,row in event_df[["eventurl","eventurlid"]].iterrows():
   fighters_table = BeautifulSoup(req.get(row.eventurl).content, 'html.parser').find("tbody")
   fights = fighters_table.find_all("tr")
   for fight in fights:

      fight_id = str(uuid.uuid4()) # Unique id for the fight 
      fighter_one_winner = [fight_id, row.eventurlid]
      fighter_two = [fight_id, row.eventurlid]
      col_vals = fight.find_all("td")

      if "win" in col_vals[0].text:
         fighter_one_winner.append("WIN")
         fighter_two.append("LOSS")
      else:
         fighter_one_winner.append("NC")
         fighter_two.append("NC")
      
      names = col_vals[1].find_all("p")
      # Name 
      fighter_one_winner.append(sp(names[0]))
      fighter_two.append(sp(names[1]))

      
      fighter_one_url = str(names[0].find("a")).split('"')[-2]
      fighter_two_url = str(names[1].find("a")).split('"')[-2]
      fighter_one_id = fighter_one_url.split("/")[-1]
      fighter_two_id = fighter_two_url.split("/")[-1]

      # Fighter id
      fighter_one_winner.append(fighter_one_id)
      fighter_two.append(fighter_two_id)

      fighter_one_winner.append(fighter_one_url)
      fighter_two.append(fighter_two_url)

      # Fight stats
      knock_downs = col_vals[2].find_all("p")
      strikes = col_vals[3].find_all("p")
      take_downs = col_vals[4].find_all("p")
      submissions = col_vals[5].find_all("p")

      fighter_one_winner.append(sp(knock_downs[0]))
      fighter_two.append(sp(knock_downs[1]))
      fighter_one_winner.append(sp(strikes[0]))
      fighter_two.append(sp(strikes[1]))
      fighter_one_winner.append(sp(take_downs[0]))
      fighter_two.append(sp(take_downs[1]))
      fighter_one_winner.append(sp(submissions[0]))
      fighter_two.append(sp(submissions[1]))

      # Class and Bonus
      class_and_bonus = col_vals[6].find("p")
      w_class = sp(class_and_bonus)
      if len(class_and_bonus.find_all("img")) == 1:
         bonus = True
      else:
         bonus = False

      fighter_one_winner.append(w_class)
      fighter_two.append(w_class)
      fighter_one_winner.append(bonus)
      fighter_two.append(False)

      method = sp(col_vals[7].find_all("p")[0])
      method_2 = sp(col_vals[7].find_all("p")[1])
      fighter_one_winner.append(method)
      fighter_two.append(method)
      fighter_one_winner.append(method_2)
      fighter_two.append(method_2)


      round_end = sp(col_vals[8].find("p"))
      fighter_one_winner.append(round_end)
      fighter_two.append(round_end)

      final_round_time = sp(col_vals[9].find("p"))
      fighter_one_winner.append(final_round_time)
      fighter_two.append(final_round_time)

      fighter_items.append(fighter_one_winner)
      fighter_items.append(fighter_two)

    

In [ ]:
header_ = [
"fightid",
"eventurlid",
"result",
"fullname",
"fighterid",
"fighterdetailsurl",
"knockdowns",
"strikes",
"takedowns",
"submission_attempts",
"weightclass",
"fightbonus",
"win_method",
"methodaction",
"finalround",
"finalround_finshtime"
]

In [ ]:
fighter_df = pd.DataFrame(data=fighter_items, columns = header_)
fighter_df

In [ ]:
fighter_df.to_csv("extraction/fight_records.csv")

NEXT STEP IS GET THE FIGHTER DETAILS

In [ ]:
def cstats(txt):
    return txt.text.replace("\n","").split(":")[1].strip()


fighter_ids = fighter_df[["fullname","fighterid","fighterdetailsurl"]].drop_duplicates("fighterid")
lastupdate = datetime.datetime.now()
fighter_details = []
for i,row in fighter_ids.iterrows():
    fighter_stats = BeautifulSoup(req.get(row.fighterdetailsurl).content, 'html.parser').find_all("div", "b-list__info-box")
    info = fighter_stats[0].find_all("li")
    stats = fighter_stats[1].find("div", "b-list__info-box-left").find_all("li")
    fighter_details.append([
            row["fighterid"],
            row["fullname"],
            cstats(info[0]),
            cstats(info[1]),
            cstats(info[2]),
            cstats(info[3]),
            cstats(info[4]),
            cstats(stats[0]),
            cstats(stats[1]),
            cstats(stats[2]),
            cstats(stats[3]),
            cstats(stats[5]),
            cstats(stats[6]),
            cstats(stats[7]),
            cstats(stats[8]),
            lastupdate])
    
    # request the above url to get fighter information
    # then we can create a players dimmension 

In [ ]:
cols = [
    "id",
    "fullname",
    "Height",
    "Weight",
    "Reach",
    "STANCE",
    "DOB",
    "SLpM",
    "StrAcc",
    "SApM",
    "StrDef",
    "TDAvg",
    "TDAcc",
    "TDDef",
    "SubAvg",
    "lastupdated"
]
df_details = pd.DataFrame(data = fighter_details, columns = cols)
df_details

In [ ]:
# Add additional data
ft_ = df_details.Height
cm = []
for ft in ft_:
    if "'" in ft:
        h_inch = float(ft.split("'")[1].replace('"',"").strip())
        h_inch += float(ft.split("'")[0]) * 12
        h_cm = round(h_inch * 2.54, 2)
        cm.append(h_cm)
    else:
        cm.append("")


wt_ = df_details.Weight
mass = []
for we in wt_:
    if "lb" in we:
        kg = round(float(we.split()[0]) * 0.453592, 2)
        mass.append(kg)
    else:
        mass.append("")


rc_ = df_details.Reach
reach = []
for rc in rc_:
    if '"' in rc:
        r_cm = round(float(rc.split('"')[0]) * 2.54, 2)
        reach.append(r_cm)
    else:
        reach.append("")

df_details["HightCM"] = cm
df_details["WeightKG"] = mass
df_details["ReachCM"] = reach

In [ ]:
df_details.to_csv("extraction/fighter_details.csv")